# First Notebook for the Capstone assignment for Week 3

### We import the required libraries

In [2]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import re
import pandas as pd
import numpy as np

### First we find the required page, parse it using BeautifulSoup, and scrap the html to get the table

In [85]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

try:
    url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
    html = urllib.request.urlopen(url, context = ctx).read()
    soup = BeautifulSoup(html, 'html.parser')
except urllib.error.HTTPError as err:
    print('***********', err.info())
    quit()
except Exception as err:    
    print(err)
    quit()

print('************Starting to scan page...*********************')

ran_list = []

code_table =  soup.find_all('table')[0]

code_table_body = code_table.tbody.find_all('tr')

for char in code_table_body:
    res = char.find_all('td')
    ran_list.extend(res)


************Starting to scan page...*********************


### The following is a function to split the borough and neighbourhood from each table cell...

In [86]:
def split_address(text):
    result = text.split('(')
    result[1] = result[1].strip(')')
    return result


### Then we create a list and for each cell, we add a dictionary containing the postal code, borough and neighbourhood

In [80]:
toronto_data_list = []

for line in ran_list:
    addr = line.p.span.text
    if addr == 'Not assigned':
        pass
    else:
        addr = split_address(addr)
        postal_code = line.text.strip()[:3]
        borough = addr[0]
        neighbourhood = addr[1]
        
        row = {'Postal_Code' : postal_code, 'Borough': borough, 'Neighbourhood': neighbourhood}
        toronto_data_list.append(row)

### Finally, we convert our data to a pandas dataframe, and sort and clean it up

In [81]:
toronto_data = pd.DataFrame.from_dict(toronto_data_list)
toronto_data = toronto_data.sort_values(by = 'Postal_Code')
toronto_data['Neighbourhood'] = toronto_data['Neighbourhood'].str.replace('/', ',')
toronto_data.head()

,Postal_Code,Borough,Neighbourhood
6,M1B,Scarborough,"Malvern , Rouge"
12,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
18,M1E,Scarborough,"Guildwood , Morningside , West Hill"
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae


In [83]:
toronto_data.shape

(103, 3)

### Last, we save our data to a csv file

In [84]:
toronto_data.to_csv('toronto_data.csv')